In [26]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import types
import toml

import date_operation as dp


db_config = toml.load('./.streamlit/secrets.toml')
host=db_config['mysql']['host']
user=db_config['mysql']['user']
password=db_config['mysql']['password']
database=db_config['mysql']['database']

# 初始化数据库链接
engine = create_engine("mysql+mysqlconnector://{user}:{pw}@{host}/{db}"
				.format(host=host, db=database, user=user, pw=password))

## 数据清洗

In [17]:
# df = pd.read_csv('./5239.txt',encoding='utf-16',delimiter=' ')
df = pd.read_excel('./11963新.xlsx')

In [18]:
# 重命名
df.columns = ['no','code','player_win','player_lose','position_win','w_l_a','w_l_1','w_l_2','w_l_3','l_l_a','l_l_1','l_l_2','l_l_3', \
            'w_w_1', 'w_w_2', 'w_w_3', 'w_w_4', 'w_w_5', 'w_w_6', 'w_w_7', 'w_w_8', \
            'l_w_1', 'l_w_2', 'l_w_3', 'l_w_4', 'l_w_5', 'l_w_6', 'l_w_7', 'l_w_8']
df.drop('no', axis=1, inplace=True)

# 去重以保证唯一性
df.drop_duplicates(subset=['code'], keep='last', inplace=True)

In [20]:
# 设定类型及索引
sql_types = {col_name: types.VARCHAR(8) for col_name in df}
sql_types['player_win'] = types.VARCHAR(20)
sql_types['player_lose'] = types.VARCHAR(20)
sql_types['position_win'] = types.Integer()
df_index = df.set_index('code', inplace=False)

# 写入数据库
df_index.to_sql('tta_pulse_code_detail',con=engine, if_exists='replace', dtype=sql_types, index=True)

In [23]:
# 源数据用户id有点问题，需要更新
sql = """
    update tta_pulse_code_detail as a
        left join tta_pulse_flat_data as b
        on a.code = b.code
    set a.player_win = b.cgeUsername
    where 1 = 1;
"""
engine.execute(sql)
sql = """
    update tta_pulse_code_detail as a
        left join tta_pulse_flat_data as b
        on a.code = b.code
    set a.player_lose = b.cgeUsername_2
    where 1 = 1;
"""
engine.execute(sql)

In [24]:
df.head(5)

,code,player_win,player_lose,position_win,w_l_a,w_l_1,w_l_2,w_l_3,l_l_a,l_l_1,...,w_w_7,w_w_8,l_w_1,l_w_2,l_w_3,l_w_4,l_w_5,l_w_6,l_w_7,l_w_8
0,TEKOLENA,Aluk,Cleveer,1,埃及艳后,诺查丹玛斯,库克,丘吉尔,孔子,古腾堡,...,0,0,斗兽场,大教堂,好莱坞,0,0,0,0,0
1,SOKOLENA,Martin_Pecheur,Aluk,0,荷马,成吉思汗,罗伯斯比尔,黛德丽,埃及艳后,埃莉诺,...,0,0,巨石阵,姬路城,红十字会,0,0,0,0,0
2,JIFOLENA,Dazzy,Martin_Pecheur,0,孔子,伊莎贝拉,库克,无,埃及艳后,成吉思汗,...,0,0,姬路城,远洋游轮,好莱坞,0,0,0,0,0
3,RUFOLENA,fu1luo,Dazzy,0,孔子,诺查丹玛斯,牛顿,无,荷马,杰士卡,...,0,0,大图书馆,泰姬陵,苏伊士运河,联合国,0,0,0,0
4,TOFOLENA,Martin_Pecheur,xuxu,0,大帝,诺查丹玛斯,拿破仑,丘吉尔,摩西,杰士卡,...,0,0,斗兽场,埃菲尔铁塔,联合国,0,0,0,0,0


## 源表建立

In [27]:
df = pd.read_sql('tta_pulse_code_detail',con=engine)

### 奇迹&领袖表

In [28]:
# 胜
df1 = df.loc[:,['code', 'player_win', 'w_w_1', 'w_w_2', 'w_w_3', 'w_w_4', 'w_w_5', 'w_w_6', 'w_w_7', 'w_w_8']]
df3 = pd.melt(df1, id_vars=["code", "player_win"],
        value_vars=['w_w_1', 'w_w_2', 'w_w_3', 'w_w_4', 'w_w_5', 'w_w_6', 'w_w_7', 'w_w_8'],
        var_name="wonder_index",
        value_name="wonder_name").sort_values('wonder_index')
df3['is_win'] = 1

df3['wonder_index'] = df3['wonder_index'].str[-1:]
df3['wonder_index'] = df3['wonder_index'].astype(int)
df3['wonder_name'] = df3['wonder_name'].astype(str)
df3 = df3.loc[df3['wonder_name'] != '0']

df2 = df.loc[:,['code', 'player_lose', 'l_w_1', 'l_w_2', 'l_w_3', 'l_w_4', 'l_w_5', 'l_w_6', 'l_w_7', 'l_w_8']]
df4 = pd.melt(df2, id_vars=["code", "player_lose"],
        value_vars=['l_w_1', 'l_w_2', 'l_w_3', 'l_w_4', 'l_w_5', 'l_w_6', 'l_w_7', 'l_w_8'],
        var_name="wonder_index",
        value_name="wonder_name").sort_values('wonder_index')
df4['is_win'] = 0

# 负
df4['wonder_index'] = df4['wonder_index'].str[-1:]
df4['wonder_index'] = df4['wonder_index'].astype(int)
df4['wonder_name'] = df4['wonder_name'].astype(str)
df4 = df4.loc[df4['wonder_name'] != '0']
df4.columns = df3.columns


# 拼接
df5 = pd.concat([df3, df4], axis=0)
df5.columns = ['code', 'player', 'wonder_no', 'wonder_name', 'is_win']

sql_types = {col_name: types.VARCHAR(8) for col_name in df}
sql_types['player'] = types.VARCHAR(20)
sql_types['wonder_no'] = types.VARCHAR(1)
sql_types['is_win'] = types.Integer()
df_index = df.set_index('code', inplace=False)
df5.set_index('code', inplace=True)

df5.to_sql('tta_pulse_wonder_detail', con=engine,if_exists='replace',index=True, dtype=sql_types)

In [29]:
df1 = df.loc[:,['code', 'player_win', 'w_l_a','w_l_1','w_l_2','w_l_3']]
df3 = pd.melt(df1, id_vars=["code", "player_win"],
        value_vars=['w_l_a','w_l_1','w_l_2','w_l_3'],
        var_name="leader_index",
        value_name="leader_name").sort_values('leader_index')
df3['is_win'] = 1
df3['leader_index'] = df3['leader_index'].str[-1:]
df3['leader_name'] = df3['leader_name'].astype(str)
df3 = df3.loc[df3['leader_name'] != '0']

df2 = df.loc[:,['code', 'player_lose', 'l_l_a','l_l_1','l_l_2','l_l_3']]
df4 = pd.melt(df2, id_vars=["code", "player_lose"],
        value_vars=['l_l_a','l_l_1','l_l_2','l_l_3'],
        var_name="leader_index",
        value_name="leader_name").sort_values('leader_index')
df4['is_win'] = 0
df4['leader_index'] = df4['leader_index'].str[-1:]
df4['leader_name'] = df4['leader_name'].astype(str)
df4 = df4.loc[df4['leader_name'] != '0']
df4.columns = df3.columns

# 拼接
df5 = pd.concat([df3, df4], axis=0)
df5.columns = df5.columns = ['code', 'player', 'leader_no', 'leader_name', 'is_win']

sql_types = {col_name: types.VARCHAR(8) for col_name in df}
sql_types['player'] = types.VARCHAR(20)
sql_types['leader_no'] = types.VARCHAR(1)
sql_types['is_win'] = types.Integer()
df_index = df.set_index('code', inplace=False)
df5.set_index('code', inplace=True)

df5.to_sql('tta_pulse_leader_detail', con=engine,if_exists='replace',index=True, dtype=sql_types)